# 03. Memória

Os modelos de linguagem são "stateless" (sem estado), ou seja, eles não lembram da conversa passada por padrão. Para criar chatbots, precisamos gerenciar o histórico da conversa e passá-lo a cada nova interação. O LangChain facilita isso.

**Objetivos:**
- Entender como funciona a memória no LCEL.
- Usar `RunnableWithMessageHistory` para gerenciar histórico automaticamente.

# Explicação Detalhada do Assunto

# 03. Memória

Bem-vindo(a) ao notebook sobre memória em LangChain! Este é um pilar fundamental para a construção de aplicações de IA generativa conversacionais, como chatbots, que precisam manter o contexto das interações para fornecer respostas relevantes e coerentes.

## Resumo Executivo

Neste notebook, exploraremos o conceito de memória em modelos de linguagem e como implementá-la utilizando LangChain. Veremos o problema da "statelessness" (ausência de estado) dos LLMs e aprenderemos a adicionar histórico de conversas às nossas aplicações, permitindo que elas "se lembrem" do que foi dito anteriormente. Utilizaremos a abordagem moderna do LangChain, com `RunnableWithMessageHistory`, para gerenciar o histórico de forma eficiente.

## Conceitos Chave

Para um melhor aproveitamento deste material, é importante ter uma compreensão básica dos seguintes conceitos:

*   **LLMs (Large Language Models):** Modelos de linguagem de grande escala, como o Gemini, que são a base da IA generativa.
*   **Statelessness:** A característica de LLMs de não manterem estado, ou seja, cada interação é independente das anteriores.
*   **Chains:** Sequências de operações (prompts, LLMs, parsers) que definem o fluxo de processamento em LangChain.
*   **Memória:** A capacidade de um sistema de IA de reter informações sobre interações passadas e usá-las para influenciar interações futuras.
*   **`RunnableWithMessageHistory`:** Uma classe do LangChain que facilita a adição de histórico de mensagens a uma chain.
*   **`ChatMessageHistory`:** Uma classe para armazenar o histórico de mensagens em memória.
*   **`MessagesPlaceholder`:** Um placeholder em um prompt que permite injetar o histórico de mensagens.

## Objetivos de Aprendizado

Ao concluir este notebook, você será capaz de:

*   Compreender o problema da falta de memória em LLMs.
*   Implementar a memória em suas aplicações LangChain utilizando `RunnableWithMessageHistory`.
*   Gerenciar o histórico de conversas com `ChatMessageHistory`.
*   Utilizar `MessagesPlaceholder` para injetar o histórico em seus prompts.
*   Construir chatbots que "se lembram" de interações passadas, fornecendo respostas mais contextuais e relevantes.

## Importância no Ecossistema LangChain

A memória é um componente essencial para a criação de aplicações de IA generativa avançadas. Sem memória, os chatbots são incapazes de manter conversas significativas e personalizadas. Este notebook fornece o conhecimento e as ferramentas necessárias para superar essa limitação, permitindo que você crie aplicações mais inteligentes e interativas. Dominar este conceito é crucial para aproveitar ao máximo o poder do LangChain e construir soluções de IA generativa de ponta. Vamos começar!

---


In [1]:
### INJECTION START ###
import os
from dotenv import load_dotenv
import sys
# Carrega .env do local ou de pastas comuns
for p in ['.', '..', 'scripts', '../scripts']:
    path = os.path.join(p, '.env')
    if os.path.exists(path):
        load_dotenv(path)
        break
if os.getenv('GOOGLE_API_KEY'):
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
### INJECTION END ###

import os
from dotenv import load_dotenv
import sys
# Autenticação automática do script
for p in ['.', '..', 'scripts', '../scripts']:
    path = os.path.join(p, '.env')
    if os.path.exists(path):
        load_dotenv(path)
        break
if os.getenv('GOOGLE_API_KEY'):
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

import os
from dotenv import load_dotenv
import sys
# Look for .env in scripts folder
for p in ['.', '..', 'scripts', '../scripts']:
    path = os.path.join(p, '.env')
    if os.path.exists(path):
        load_dotenv(path)
        break
if os.getenv('GOOGLE_API_KEY'):
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

import os
from dotenv import load_dotenv
load_dotenv()

# !pip install -qU langchain langchain-openai langchain-community python-dotenv # Script-patched

False

In [2]:
import os
try:
    from google.colab import userdata
except ImportError:
    userdata = None
import getpass

try:
    pass # Script-patched
except:
    pass # Script-patched

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.9.6). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/auth/__init__.py:54: FutureWarning: You are using a Python version 3.9 past its end of life. Google will update google-auth with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade your Python

/Users/naubergois/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/langchain_google_genai/chat_models.py:47: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  from google.generativeai.caching import CachedContent  # type: ignore[import]


## 1. O Problema da Falta de Memória

Vamos ver como o modelo se comporta sem memória.

In [4]:
chain = ChatPromptTemplate.from_template("{input}") | llm | StrOutputParser()

# Primeira interação
print(chain.invoke({"input": "Oi, meu nome é Nauber."}))

# Segunda interação
print(chain.invoke({"input": "Qual é o meu nome?"}))

Olá Nauber, tudo bem? É um prazer te conhecer! Em que posso te ajudar hoje?


Eu sou um modelo de linguagem grande, treinado pelo Google. Eu não tenho nome.


Ele provavlemente dirá que não sabe, pois cada chamada é independente.

## 2. Adicionando Histórico com `RunnableWithMessageHistory`

Essa é a forma recomendada no LCEL moderno. Precisamos de uma classe para armazenar o histórico (aqui usaremos `ChatMessageHistory` em memória, mas poderia ser num banco de dados).

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Dicionário para guardar os históricos de diferentes sessões (session_ids)
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

Agora criamos o prompt aceitando um `MessagesPlaceholder` para injetar o histórico.

In [6]:
from langchain_core.prompts import MessagesPlaceholder

prompt_with_history = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente prestativo."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

runnable = prompt_with_history | llm | StrOutputParser()

# Envolvemos a chain original com a capacidade de histórico
with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

## 3. Testando a Memória

Agora vamos conversar passando um `session_id`.

In [7]:
# Configurando o ID da sessão
config = {"configurable": {"session_id": "sessao_do_nauber"}}

response1 = with_message_history.invoke(
    {"input": "Oi, meu nome é Nauber."}, 
    config=config
)
print(f"Resposta 1: {response1}")

response2 = with_message_history.invoke(
    {"input": "Qual é o meu nome?"}, 
    config=config
)
print(f"Resposta 2: {response2}")

Resposta 1: Olá Nauber, é um prazer conhecê-lo! Em que posso ajudá-lo hoje?


Resposta 2: Seu nome é Nauber.


## 4. Chats Diferentes (Session IDs)

Se mudarmos o `session_id`, ele não lembrará.

In [8]:
config_novo = {"configurable": {"session_id": "sessao_nova"}}

response3 = with_message_history.invoke(
    {"input": "Qual é o meu nome?"}, 
    config=config_novo
)
print(f"Resposta 3 (Sessão Nova): {response3}")

Resposta 3 (Sessão Nova): Eu não tenho acesso a informações pessoais sobre você, então eu não sei o seu nome.


## Conclusão

Neste notebook, aprendemos a manter o estado da conversa usando `RunnableWithMessageHistory` e `ChatMessageHistory`.

No próximo notebook, vamos explorar **Chains** mais complexas.